In [60]:
import os
import sys
import fnmatch
import pandas as pd
import operator
import shutil
import re
import javalang
def create_dev_test_train_split_and_vocabulary(root_path, 
                                               train_output, 
                                               vocabFile,
                                               combine
                                              ):

    train_file = ''

    word_counts = dict()
    
    for root, dirnames, filenames in os.walk(root_path):
        for filename in fnmatch.filter(filenames, '*.csv'):
            path = "csv/trimedValues.csv"
            train_file = path
            dataframe = pd.read_csv(path, na_filter = False)
            for i,data in dataframe.iterrows():

                d = (data["comment"]).split()
                c = (data["code"]).split()
#                 print("comment: ", d)
#                 print("code: ", c)
#                 d = padding(d)
#                 c = padding(c)
                s = []
                s =  d+[SPL]+c # comment and code together
#                 print(s)
                add_counts(word_counts, s)
#             print(word_counts)

    vocabulary = build_vocabulary(word_counts)
#     print("-- VOCABULARY BEGIN --")
#     print(vocabulary)
#     print("-- VOCABULARY END --")
    write_vocabulary(vocabulary, vocabFile)

    write_processed_dataset(train_file, train_output,vocabFile, combine)

def write_processed_dataset(input_file, output_file, vocabFile, combine):
    df = pd.DataFrame()
    word_vocabulary = read_vocabulary(vocabFile)
    dataframe = pd.read_csv(input_file, na_filter = False)
    for i,d in dataframe.iterrows():
        comment = []
        code = []
        label = 1 if d["non-information"] == "yes" else 0
#         for token in padding((d["comment"]).split()):
# #             print(token)
# #             print(word_vocabulary.get(token,0))
#             comment.append(word_vocabulary.get(token,0))
#         for token in padding((d["code"]).split()):
# #             print(token)
# #             print(word_vocabulary.get(token,0))
#             code.append(word_vocabulary.get(token,0))
        for token in (d["comment"]).split():
            comment.append(word_vocabulary.get(token,0))
        for token in (d["code"]).split():
            code.append(word_vocabulary.get(token,0))
        if combine:
            data = comment + [1] +code
#             data = comment +code
#             data = padding_number(data)
            print(data)
            
            da = [{
                "data":data,
                "label":label,
            }]
            
            df = df.append(da,ignore_index=True,sort=False)
        else:
            da = [{
                "comment":comment,
                "code":code,
                "label":label,
            }]
            print(da)
            df = df.append(da,ignore_index=True,sort=False)

    df.to_csv(output_file, index=False)

In [61]:
delimiters = "#", ".",",","<b>","</b>","-",":","<br>","_","?"," ",";"
def splitComment(string,delimiters = delimiters, maxsplit=0):
## replace all web https to https
    for l in string.split():
        if l.startswith("https"):
            string = string.replace(l,"https")
    ## split string by delimiters
    regexPattern = '|'.join(map(re.escape, delimiters))
    result =  re.split(regexPattern, string, maxsplit)
    ## split string by uppercase
    f = []
    for r in result:
        p =  re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', r)
#         print(p)
        for q in p.split():
            if q != "":
                f.append(q)
    return f
def tokenizeJavaCode(code):
    result = []
    try:
        tokens = list(javalang.tokenizer.tokenize(code))
        for token in tokens:
            result.append(token.value)
        return result
    except:
        return list(code)
#TODO stopwords
STOPWORDS_RAW=["(?i)or$", "(?i)and$", "(?i)i$"]

def iterable_to_dict(arr):
#     print("-- DEBUG BEGIN --")
#     print(dict((x.strip(), i) for (i, x) in enumerate(arr)))
#     print("-- DEBUG END --")
    return dict((x.strip(), (i)) for (i, x) in enumerate(arr))

def read_vocabulary(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        return iterable_to_dict(f.readlines())
    
def add_counts(word_counts, line):
    stopreg = []
    for rawregex in STOPWORDS_RAW:
        stopreg.append(re.compile(rawregex))
#     print(stopreg)
    for w in line:  
        if any(regex.match(w) for regex in stopreg):
#             print("Stopword:",w)
            continue
        word_counts[w] = word_counts.get(w, 0) + 1
        
END = "</S>"
# SPL = "<SPL>"
PAD = "<PAD>"
# NUM = "<NUM>"
UNK = "<NUK>"
MAX_WORD_VOCABULARY_SIZE = 100000
MIN_WORD_COUNT_IN_VOCAB = 2
MAX_SEQUENCE_LEN = 45


def padding(sentences, padding_size = MAX_SEQUENCE_LEN):
    result = []
    if len(sentences) >= padding_size:
        result = sentences[:padding_size]
    else:
        result = sentences
        for i in range(padding_size-len(sentences)):
            result.append(PAD)
    return result

def padding_number(sentences, padding_size = MAX_SEQUENCE_LEN):
    result = []
    if len(sentences) >= padding_size:
        result = sentences[:padding_size]
    else:
        result = sentences
        for i in range(padding_size-len(sentences)):
            result.append(1)
    return result

def dump(d, path):
    with open(path, 'w') as f:
        for s in d:
            f.write("%s\n" % repr(s))
        
def write_vocabulary(vocabulary, file_name):
    vocabulary.insert(0,UNK)
    if END not in vocabulary:
        vocabulary.append(END)

    print("Vocabulary size (write_vocabulary): %d" % len(vocabulary))

    with open(file_name, 'w', encoding='utf-8') as f:
        f.write("\n".join(vocabulary))
        
def build_vocabulary(word_counts):
#     for wc in reversed(sorted(word_counts.items(), key=operator.itemgetter(1))):
#         print(wc[0], " ", wc[1])
    return [wc[0] for wc in reversed(sorted(word_counts.items(), key=operator.itemgetter(1))) if wc[1] >= MIN_WORD_COUNT_IN_VOCAB and wc[0] != UNK][:MAX_WORD_VOCABULARY_SIZE] # Unk will be appended to end

In [62]:
root_path = "csv"
vocabFile = "csv/vocab.txt"
create_dev_test_train_split_and_vocabulary(root_path,"csv/training_data.csv",vocabFile,True)

Vocabulary size (write_vocabulary): 2269
[1425, 623, 445, 35, 58, 550, 726, 398, 397, 257, 1424, 26, 396, 1424, 65, 68, 1423, 398, 397, 161, 65, 68, 94, 1, 9, 64, 25, 62, 65, 68, 26, 396, 26, 396, 65, 68, 65, 68, 65, 68, 94]
[47, 19, 725, 724, 26, 857, 40, 59, 4, 127, 1, 70, 395, 131, 227, 16, 857, 47]
[2267, 102, 8, 1057, 200, 52, 43, 490, 549, 52, 1, 314, 622, 288, 1056, 52, 69, 490, 549, 52]
[548, 7, 86, 1055, 621, 2, 358, 56, 32, 1, 38, 85, 1422, 358, 61, 51, 51]
[138, 199, 1, 1421, 394, 216, 215, 47, 199]
[99, 7, 136, 3, 57, 24, 43, 393, 46, 7, 96, 3, 856, 8, 24, 96, 75, 57, 144, 1, 97, 5, 24, 2, 75, 34]
[6, 24, 1, 12, 110, 6, 6, 24, 26, 16, 6, 26, 119, 6, 19]
[93, 2266, 2265, 0, 2264, 74, 18, 547, 45, 10, 2263, 126, 93, 74, 99, 1420, 89, 22, 3, 17, 547, 10, 2262, 1]
[256, 1054, 546, 91, 392, 1053, 723, 722, 2261, 23, 1, 7, 3, 287, 545, 5, 0, 545, 5, 544]
[855, 30, 1419, 2, 238, 444, 107, 1, 12, 198, 122, 391, 2, 49, 107]
[2, 2260, 189, 2259, 28, 721, 106, 1418, 443, 2, 1052, 5, 0

[1356, 29, 2174, 118, 690, 1, 418, 371, 118, 126]
[4, 11, 381, 43, 5, 24, 47, 1, 9, 11, 4, 47, 46]
[1419, 2, 18, 2173, 2, 444, 107, 74, 18, 373, 156, 689, 218, 30, 349, 2, 20, 1, 2, 49, 107, 30, 391]
[19, 128, 159, 128, 1, 41]
[28, 106, 17, 819, 52, 213, 515, 2, 1009, 810, 2, 142, 2172, 213, 2171, 0, 213, 478, 8, 279, 1, 7]
[2170, 2169, 2168, 2, 2167, 595, 210, 809, 2166, 1355, 1, 11, 1354, 34]
[60, 68, 487, 144, 12, 196, 1353, 1, 20, 427, 196, 2165, 13, 210, 20]
[28, 104, 66, 0, 417, 520, 1, 20, 104, 95, 808, 382, 95, 104, 817, 13, 210, 20]
[594, 20, 180, 214, 29, 40, 59, 8, 223, 0, 807, 8, 224, 302, 286, 1, 9, 14, 40, 59, 20, 180, 262, 165, 374, 20, 40, 20, 180]
[93, 331, 71, 7, 71, 3, 78, 31, 484, 2164, 515, 125, 63, 21, 8, 31, 4, 125, 124, 303, 71, 303, 4, 48, 688, 21, 1]
[93, 806, 352, 11, 46, 544, 96, 22, 17, 1008, 2163, 2162, 28, 74, 2161, 687, 281, 1]
[15, 319, 105, 686, 8, 137, 467, 2, 115, 12, 134, 306, 2, 1352, 1, 9, 14, 115, 12, 134, 521, 15]
[93, 222, 2160, 485, 2, 88, 149

[14, 12, 1393, 0, 2059, 1392, 199, 1, 9, 14, 955, 0, 301, 6]
[2058, 418, 89, 136, 54, 769, 433, 453, 8, 182, 21, 31, 1, 7, 13, 169, 4, 31, 683, 182, 21, 31]
[60, 73, 2057, 55, 1, 73, 55, 13, 73, 127, 135, 55]
[768, 65, 68, 7, 86, 1408, 65, 68, 44, 1, 70, 40, 30, 94, 375, 44, 94, 44, 2056, 94]
[1, 9, 64, 170, 10, 118, 117, 146, 503, 503]
[30, 124, 16, 12, 195, 44, 1, 119, 30, 609, 16, 44, 16]
[75, 207, 574, 106, 17, 456, 527, 36, 33, 329, 1, 20, 11, 442, 13, 210, 20]
[1255, 21, 54, 2055, 296, 108, 140, 316, 113, 246, 61, 1, 110, 23, 6, 23, 32, 21, 222, 655, 21, 362, 655, 113, 246, 61]
[12, 2054, 21, 1254, 83, 2, 18, 90, 2053, 954, 89, 144, 78, 465, 20, 2052, 2, 78, 19, 8, 2051, 390, 1, 12, 23, 6, 6, 21]
[1008, 243, 441, 8, 58, 77, 337, 177, 2050, 77, 3, 486, 10, 1253, 146, 1, 9, 14, 57, 77, 337, 165, 77, 337]
[245, 10, 61, 405, 2049, 29, 767, 1, 7, 94, 4, 150, 44, 150, 767]
[160, 17, 12, 6, 1, 33, 133, 4, 422, 30, 80, 44]
[7, 28, 184, 22, 75, 22, 35, 24, 20, 1, 7, 2, 75, 34]
[179, 404, 

[110, 837, 543, 97, 5, 164, 1, 38, 81, 19, 97, 5, 24, 97, 5, 164, 13, 130, 1208, 584, 1207, 97, 5, 1937, 46]
[774, 445, 35, 1, 38, 25, 645, 265, 73, 16, 152, 26, 16, 152, 26, 16]
[389, 670, 133, 79, 57, 1, 41, 197]
[28, 54, 17, 1351, 136, 3, 256, 466, 251, 136, 651, 91, 392, 449, 2, 225, 282, 1, 7, 241, 755, 651, 91, 392, 449, 1206]
[425, 449, 1, 451, 495, 451, 495, 13, 451, 495, 449, 451, 495, 168, 273, 250]
[283, 19, 96, 28, 108, 57, 212, 1, 675, 39, 4, 36, 33, 39, 152, 107, 46, 1205, 34]
[170, 529, 49, 10, 146, 2, 291, 919, 1936, 428, 5, 3, 287, 1, 155, 77, 510, 49, 22, 5, 15, 104, 789, 15, 104, 169, 15, 104, 484, 1204]
[22, 300, 18, 89, 918, 67, 3, 34, 12, 501, 7, 22, 1240, 608, 10, 1935, 1, 7, 163, 34]
[776, 101, 1053, 1, 11, 1354, 153, 648]
[417, 1203, 2, 366, 10, 144, 219, 1, 12, 23, 6, 6, 21]
[93, 30, 71, 331, 71, 199, 154, 125, 124, 71, 30, 263, 21, 34, 125, 388, 31, 71, 13, 125, 388, 31, 31, 484, 236, 169, 183, 48, 13, 31, 7, 71, 30, 263, 21, 34, 51, 4, 71, 143, 30, 124, 71, 

[2, 1842, 12, 1841, 0, 12, 1840, 28, 237, 506, 267, 304, 1, 88, 614, 207, 574]
[49, 1839, 344, 1230, 414, 12, 56, 113, 11, 1, 9, 64, 25, 49, 97, 414, 23, 32, 65, 32, 65, 11, 273, 113, 16, 273, 16]
[481, 1]
[274, 1838, 35, 1165, 401, 0, 60, 70, 1165, 1837, 12, 349, 1, 38, 25, 1164, 70]
[26, 1178, 162, 2, 1836, 229, 1835, 1, 11, 903, 450, 1834, 1833, 1832]
[750, 281, 12, 1831, 1163, 172, 1162, 16, 1, 13, 301, 1163, 172, 1162, 42]
[93, 62, 1830, 560, 8, 102, 11, 903, 637, 13, 11, 903, 472, 127, 135, 102, 142, 176, 416, 2, 276, 1161, 653, 1829, 637, 450, 102, 252, 110, 712, 712, 44, 417, 1189, 1828, 1827, 637, 450, 472, 394, 100, 19, 26, 637, 2, 11, 1, 41, 13, 203, 127, 135, 1161, 63]
[461, 11, 1826, 12, 188, 123, 1, 845]
[131, 671, 1825, 1, 260, 1026, 13, 260]
[99, 7, 28, 74, 1160, 1159, 1158, 11, 10, 329, 1, 85, 1160, 151, 4, 85, 1020, 69, 524, 23, 151, 1824, 1159, 1158]
[1216, 86, 2, 406, 21, 116, 1, 15, 174, 62, 199, 15, 0, 115]
[1823, 63, 27, 3, 902, 12, 536, 509, 71, 124, 2, 186, 233

[1136, 151, 5, 0, 318, 803, 1004, 803, 1004, 83, 2, 18, 1750, 29, 79, 511, 736, 735, 423, 633, 150, 69, 136, 433, 310, 1135, 105, 120, 736, 735, 423, 633, 150, 3, 224, 255, 324, 150, 61, 633, 150, 3, 184, 50, 18, 1749, 0, 29, 102, 10, 5, 151, 473, 50, 18, 1748, 0, 102, 54, 1044, 206, 105, 67, 2, 138, 5, 3, 759, 2, 736, 735, 423, 633, 150, 255, 56, 151, 3, 287, 10, 1023, 2, 61, 138, 5, 102, 10, 224, 138, 5, 882, 2, 1747, 784, 8, 177, 105, 905, 35, 1220, 1746, 806, 89, 150, 61, 633, 150, 3, 1134, 10, 289, 436, 2, 108, 138, 5, 3, 1134, 10, 929, 312, 696, 67, 98, 18, 56, 45, 52, 12, 1745, 620, 736, 735, 423, 138, 67, 2, 1744, 61, 138, 150, 1, 9, 14, 423, 194]
[19, 6, 234, 10, 196, 3, 547, 10, 149, 242, 1, 189, 262, 6, 234]
[12, 756, 261, 3, 17, 19, 468, 96, 674, 568, 2, 106, 222, 485, 2, 785, 36, 33, 47, 40, 1, 73, 261, 69, 1171]
[1133, 225, 133, 2, 181, 1]
[14, 3, 1743, 1132, 79, 940, 1, 757, 431, 1210]
[298, 123, 746, 29, 363, 86, 1138, 383, 10, 303, 223, 593, 563, 229, 246, 223, 221, 15

[1645, 0, 213, 478, 99, 7, 136, 54, 67, 1644, 10, 474, 7, 96, 22, 3, 1643, 17, 509, 474, 96, 28, 74, 687, 162, 1, 7, 874, 113, 1642, 874, 113, 733, 733, 8]
[27, 1641, 49, 61, 1640, 2, 19, 162, 1045, 1639, 105, 226, 35, 111, 1638, 316, 2, 13, 330, 1, 9, 25, 645, 182, 1045]
[1637, 89, 28, 342, 1636, 2, 80, 5, 10, 507, 2, 506, 157, 1, 535, 13, 53, 5, 80, 5]
[1133, 225, 133, 2, 277, 1]
[237, 18, 590, 156, 1049, 8, 40, 249, 22, 129, 17, 271, 1, 1197, 170, 512]
[893, 24, 10, 196, 28, 147, 2, 1635, 1, 164, 24]
[1096, 2, 708, 1, 11, 1033, 2, 708, 1096, 2, 1634, 1633, 1632]
[106, 17, 49, 483, 5, 1, 391, 4, 32, 65, 265, 32, 5]
[28, 142, 2, 131, 6, 82, 263, 221, 407, 249, 1631, 328, 338, 3, 184, 116, 1, 6, 82, 263, 51, 6, 82, 263, 51, 127, 135, 6, 82, 263, 51, 1630, 44, 114, 1629, 40]
[88, 492, 55, 153, 105, 1628, 758, 362, 1, 231, 143, 492, 55, 153, 69, 19, 13, 61, 51, 4, 80, 55, 153]
[60, 555, 73, 12, 238, 53, 711, 1, 562, 100, 555, 73, 1026, 13, 562, 100]
[10, 196, 8, 930, 48, 160, 1627, 8, 18

[145, 219, 8, 141, 400, 54, 1511, 35, 80, 713, 1, 9, 141, 400, 4, 141, 400]
[1123, 0, 482, 329, 214, 29, 53, 21, 2, 175, 293, 359, 95, 508, 58, 264, 151, 359, 95, 1, 38, 25, 167, 264, 2, 175, 264, 151, 359, 95, 359, 95]
[1510, 53, 21, 35, 32, 293, 534, 7, 185, 86, 50, 4, 1091, 7, 1139, 1055, 621, 2, 380, 21, 0, 22, 50, 18, 1011, 45, 608, 7, 197, 42, 50, 18, 1011, 45, 534, 1, 9, 25, 380, 85, 534]
[27, 3, 226, 35, 36, 33, 159, 7, 32, 459, 42, 3, 1423, 79, 86, 332, 91, 7, 22, 3, 339, 0, 985, 166, 187, 2, 324, 166, 252, 1, 9, 25, 170, 91, 81, 123, 91]
[27, 3, 226, 35, 36, 33, 159, 105, 86, 621, 2, 60, 13, 6, 23, 6, 13, 6, 1, 9, 25, 229, 6, 81, 23, 6, 23, 6]
[19, 162, 37, 275, 12, 32, 1, 38, 25, 645, 37, 275]
[355, 6, 82, 45, 593, 132, 10, 648, 100, 7, 6, 82, 256, 208, 319, 704, 464, 89, 559, 129, 17, 230, 316, 168, 2, 399, 82, 255, 253, 56, 6, 293, 6, 6, 2, 124, 1, 9, 25, 62, 0, 124, 23, 6, 6]
[27, 646, 56, 6, 0, 1509, 22, 172, 40, 10, 70, 7, 6, 82, 3, 319, 22, 3, 56, 269, 1417, 1, 9, 25, 

[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41, 185]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41, 185]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41, 185]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 34]
[37, 72, 27, 76, 1, 41, 185]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1]
[37, 72, 27, 76, 1, 41, 